In [3]:
import os
import sys
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

project = 'HVIDEO'
path_to_raw = '../data/'+project+'/raw/'
path_to_processed = '../data/'+project+'/'

In [4]:
# this cell is required for colab only

from google.colab import drive
drive.mount('/content/drive')

path_to_raw = './drive/MyDrive/Science/Multidomain RecSys/' + path_to_raw[3:]
path_to_processed = './drive/MyDrive/Science/Multidomain RecSys/source/iz-dev/' + path_to_processed[3:]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
files = ['traindata.txt','validdata.txt','testdata.txt']
domains = {'E':[],
           'V':[]
          }

all_data = pd.DataFrame(columns=['user_id','history'])
for f in files:

    string_input = []
    with open(path_to_raw+f, 'r') as myfile:
        print('loading file: %s...'%(path_to_raw+f))

        for line in tqdm(myfile.readlines()):
            user_id = line.split('\t')[0]
            line = line.replace('||\t', '\n'+user_id+'\t').replace('||','')
            string_input.append(line)

    string_input = ''.join(string_input)
    df = pd.read_csv(StringIO(string_input), sep='\t', header=None, names=['user_id','history'])

    all_data = pd.concat([all_data, df], axis=0)

all_data['domain'] = all_data['history'].apply(lambda x: x.split('|')[0][0])
all_data['item_id'] = all_data['history'].apply(lambda x: x.split('|')[0][1:])
all_data['timestamp'] = all_data['history'].apply(lambda x: x.split('|')[3])

all_data.drop('history', axis=1, inplace=True)
all_data.drop_duplicates(inplace=True)

loading file: ./drive/MyDrive/Science/Multidomain RecSys/data/HVIDEO/raw/traindata.txt...


100%|██████████| 102182/102182 [00:01<00:00, 86556.40it/s]


creating dataframe...
loading file: ./drive/MyDrive/Science/Multidomain RecSys/data/HVIDEO/raw/validdata.txt...


100%|██████████| 18966/18966 [00:00<00:00, 68487.78it/s]


creating dataframe...
loading file: ./drive/MyDrive/Science/Multidomain RecSys/data/HVIDEO/raw/testdata.txt...


100%|██████████| 13201/13201 [00:00<00:00, 41108.69it/s]


creating dataframe...


In [ ]:
def create_all_data(
    df,
    output_path,
    output_name='all_data.txt',
    convert_dtypes=False,
    save_result=False,
    return_result=True,
):
    # part 1
    data = []
    for _, row in tqdm(df.iterrows()):
        data.append({
            'user_id': int(row.user_id) if convert_dtypes else row.user_id,
            'item_id':  int(row.item_id) if convert_dtypes else row.item_id,
            'timestamp': int(row.timestamp) if convert_dtypes else row.timestamp
        })


    # part 2
    user_history = defaultdict(list)
    item_history = defaultdict(list)

    for row in tqdm(data):
        user_raw_id = row['user_id']
        item_raw_id = row['item_id']
        interaction_timestamp = row['timestamp']

        user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})
        item_history[item_raw_id].append({'user_id': user_raw_id, 'timestamp': interaction_timestamp})


    # part 3
    is_changed = True
    threshold = 5
    good_users = set()
    good_items = set()

    while is_changed:
        old_state = (len(good_users), len(good_items))

        good_users = set()
        good_items = set()

        for user_id, history in user_history.items():
            if len(history) >= threshold:
                good_users.add(user_id)

        for item_id, history in item_history.items():
            if len(history) >= threshold:
                good_items.add(item_id)

        user_history = {
            user_id: list(filter(lambda x: x['item_id'] in good_items, history))
            for user_id, history in user_history.items()
        }

        item_history = {
            item_id: list(filter(lambda x: x['user_id'] in good_users, history))
            for item_id, history in item_history.items()
        }

        new_state = (len(good_users), len(good_items))
        is_changed = (old_state != new_state)
        print(old_state, new_state)


    # part 4
    user_mapping = {}
    item_mapping = {}
    tmp_user_history = defaultdict(list)
    tmp_item_history = defaultdict(list)

    for user_id, history in tqdm(user_history.items()):
        processed_history = []

        for filtered_item in history:
            item_id = filtered_item['item_id']
            item_timestamp = filtered_item['timestamp']

            processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
            item_mapping[item_id] = processed_item_id

            processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})

        if len(processed_history) >= threshold:
            processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
            user_mapping[user_id] = processed_user_id

            tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])


    for item_id, history in tqdm(item_history.items()):
        processed_history = []

        for filtered_user in history:
            user_id = filtered_user['user_id']
            user_timestamp = filtered_user['timestamp']

            processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
            user_mapping[user_id] = processed_user_id

            processed_history.append({'user_id': processed_user_id, 'timestamp': user_timestamp})

        if len(processed_history) >= threshold:
            processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
            item_mapping[item_id] = processed_item_id

            tmp_item_history[processed_item_id] = sorted(processed_history, key=lambda x: x['timestamp'])

    user_history = tmp_user_history
    item_history = tmp_item_history

    # part 5
    if save_result:
        with open(output_path+'/'+output_name, 'w') as f:
            for user_id, item_history in user_history.items():
                f.write(' '.join([str(user_id)] + [
                    str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
                ]))
                f.write('\n')

    if return_result:
        return user_history, item_history
    return None, None

In [ ]:
for d in tqdm(domains):
    new_dir = path_to_processed + d
    if not os.path.isdir(new_dir):
        os.makedirs(new_dir)

    df = all_data[all_data['domain']==d].drop('domain', axis=1).reset_index(drop=True)

    user_history, item_history = create_all_data(df, new_dir)
    for output_name in ['all_data.txt','train_new.txt','validation_new.txt','test_new.txt']:
        with open(new_dir+'/'+output_name, 'w') as f:
                for user_id, item_history in user_history.items():
                    f.write(' '.join([str(user_id)] + [
                        str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
                    ]))
                    f.write('\n')

  0%|          | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1328it [00:00, 12245.25it/s]
2673it [00:00, 12927.51it/s]
4055it [00:00, 13325.55it/s]
5528it [00:00, 13870.16it/s]
6917it [00:00, 13157.70it/s]
8342it [00:00, 13511.99it/s]
9725it [00:00, 13612.05it/s]
11222it [00:00, 14034.18it/s]
12659it [00:00, 14137.17it/s]
14126it [00:01, 14298.20it/s]
15558it [00:01, 14184.95it/s]
16979it [00:01, 14072.83it/s]
18393it [00:01, 14092.04it/s]
19886it [00:01, 14341.83it/s]
21321it [00:01, 13915.72it/s]
22716it [00:01, 13894.34it/s]
24200it [00:01, 14171.07it/s]
25624it [00:01, 14188.96it/s]
27046it [00:01, 14196.89it/s]
28545it [00:02, 14430.86it/s]
30052it [00:02, 14619.51it/s]
31527it [00:02, 14657.35it/s]
33028it [00:02, 14761.04it/s]
34505it [00:02, 14666.38it/s]
35973it [00:02, 14575.21it/s]
37431it [00:02, 13843.58it/s]
38859it [00:02, 13967.90it/s]
40357it [00:02, 14262.15it/s]
41843it [00:02, 14437.37it/s]
43316it [00:03, 14522.22it/s]
44772it [00:03, 13252.90it/s]
46121it [00:03, 1163

(0, 0) (13713, 3306)
(13713, 3306) (13710, 3306)
(13710, 3306) (13710, 3306)



100%|██████████| 13714/13714 [00:00<00:00, 29333.91it/s]

 50%|█████     | 1/2 [00:31<00:31, 32.00s/it]
0it [00:00, ?it/s]
1534it [00:00, 15336.87it/s]
3068it [00:00, 13857.16it/s]
4483it [00:00, 13982.17it/s]
5953it [00:00, 14252.75it/s]
7383it [00:00, 13658.63it/s]
8755it [00:00, 13626.32it/s]
10176it [00:00, 13809.93it/s]
11561it [00:00, 13710.12it/s]
12935it [00:00, 13594.27it/s]
14362it [00:01, 13796.80it/s]
15832it [00:01, 14067.89it/s]
17241it [00:01, 13724.94it/s]
18721it [00:01, 14041.17it/s]
20142it [00:01, 14090.77it/s]
21553it [00:01, 13621.18it/s]
22964it [00:01, 13760.30it/s]
24443it [00:01, 14062.50it/s]
25853it [00:01, 14052.14it/s]
27261it [00:01, 13960.94it/s]
28659it [00:02, 13877.02it/s]
30061it [00:02, 13917.12it/s]
31532it [00:02, 14151.84it/s]
33001it [00:02, 14311.30it/s]
34433it [00:02, 14109.58it/s]
35845it [00:02, 13508.36it/s]
37266it [00:02, 13709.27it/s]
38745it [00:02, 14022.84it/s]
40152it [00:02, 14013.49it/s]
41617it [00:02, 14199.30it/s]
43040it [00:

(0, 0) (13689, 3540)
(13689, 3540) (13649, 3538)
(13649, 3538) (13649, 3534)
(13649, 3534) (13649, 3534)



100%|██████████| 13714/13714 [00:00<00:00, 17545.92it/s]

100%|██████████| 2/2 [00:58<00:00, 29.11s/it]
